In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import pathlib
main_path= pathlib.Path('..')
print (main_path.resolve())
sys.path.insert(0, str(main_path.resolve()))

In [ ]:
import stash_ai.config as stash_ai_config

stash_ai_config.config_file= main_path.joinpath('config.json')
print(stash_ai_config.config_file.resolve())
load_config()
config

In [ ]:
from PIL import Image
from deepface import DeepFace
import pandas as pd
import numpy as np
import pathlib
from IPython.display import display
from matplotlib import pyplot as plt, image
import string
from dataclasses import dataclass, field
from typing import List, Dict
import cv2
import matplotlib.patches as patches
import json
from imagehash import phash
%matplotlib inline 


In [ ]:
safechars = string.ascii_lowercase + string.ascii_uppercase + string.digits + '.-_'
def to_safechars(input: str) -> str:
    return ''.join([c for c in input if c in safechars])

In [ ]:
from stash_ai.config import config

In [ ]:
config.data_dir

In [ ]:
base_dir= config.base_dir
assets_dir= config.data_dir
video_samples_dir= assets_dir.joinpath('video_samples')
video_images_dir= assets_dir.joinpath('video_samples_frames')
extracted_faces= assets_dir.joinpath('extracted_faces')
if not extracted_faces.exists():
    extracted_faces.mkdir(parents=True)
print(f"base_dir : {base_dir.resolve()}")
print(f"assets_dir : {assets_dir.resolve()}")
print(f"video_samples_dir : {video_samples_dir.resolve()}")
print(f"video_images_dir : {video_images_dir.resolve()}")

In [ ]:
@dataclass
class VideoConfig:
    video_file: pathlib.Path
    safe_name: str
    frame_path: str
    images: List[pathlib.Path]= field(default_factory=lambda: [])
    def image_count(self):
        return len(self.images)
    def __repr__(self):
        return f"VideoConfig (Filename: {self.video_file.name} SafeName: {self.safe_name} Frame path: {self.frame_path.relative_to(assets_dir)} Image count {self.image_count()})"

In [ ]:
videos={}
for f in video_samples_dir.iterdir():
    if f.is_file():
        video= VideoConfig(video_file=f, safe_name=to_safechars(f.stem), frame_path=video_images_dir.joinpath(to_safechars(f.stem)))
        if video.frame_path.exists():
            for f in video.frame_path.iterdir():
                if f.is_file():
                    video.images.append(f)
        print(f"{video.safe_name}")
        videos[video.safe_name]=video


In [ ]:
name="ClosingTimeSample"
video= videos[name]
print(video)

In [ ]:
import utils.scene

In [ ]:
detector= 'yolov8'
face_expand= 30
minimum_confidence= 0.8
p= 'C:/Users/EricGuinois/VSCODE/stash-ai-helper/local_assets/video_samples_frames/ClosingTimeSample/ClosingTimeSample_12.jpg'
f= pathlib.Path(p)
img_metadata= ImageAnalysis(f, f.stem, True)
metadata: ImageAnalysis= img_metadata
results= DeepFace.analyze(metadata.get_numpy(), actions=["age", "gender", "race"], enforce_detection=True, expand_percentage=face_expand)
results



In [ ]:
plt.imshow(metadata.get_numpy())
plt.show()

In [ ]:
for face_result in results:
    face= Face(x=face_result["region"]["x"],
               y= face_result["region"]["y"],
               w= face_result["region"]["w"],
               h= face_result["region"]["h"],
               age= face_result["age"],
               gender= face_result["dominant_gender"],
               gender_confidence=face_result["gender"][face_result["dominant_gender"]],
               race=face_result["dominant_race"],
               race_confidence=face_result["race"][face_result["dominant_race"]],
               confidence=face_result["face_confidence"]
               )
    other_face: Face
    for other_face in metadata.faces:
        if face.overlap(other_face):
            other_face.overlapping= True
            face.overlapping= True
    metadata.faces.append(face)
    print(face)

In [ ]:
metadata.get_numpy_with_overlay().flags
metadata.get_numpy_with_overlay().setflags(write=1)

In [ ]:
if metadata.sample:
    for face in metadata.faces:
        if face.overlapping:
            color= 'red'
            lw= 4
        else:
            color='green'
            lw= 2
        print(type(metadata.get_numpy_with_overlay()))
        cv2.rectangle(metadata.get_numpy_with_overlay(), face.get_top_left().to_tuple(), face.get_bottom_right().to_tuple(), (36,255,12), 1)
plt.imshow(metadata.get_numpy_with_overlay())
plt.show()
        #cv2.rectangle(metadata.get_numpy_with_overlay(), face.get_top_left(), face.get_bottom_right(), color, lw)
        #cv2.putText(metadata.get_numpy_with_overlay(), f"{face.confidence*100}%", cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, lw)
        

In [ ]:
detector= 'yolov8'
color='yellow'
limits= []
limits= ["untagged"]
limits= ["ClosingTimeSample"]
for video_name, video in videos.items():
    if limits is not None and video_name not in limits:
        continue
    for index, image_path in enumerate(video.images):
        print(f"{index}: {image_path.name} : {image_path.resolve()}")
        image= Image.open(image_path)
        img= np.asarray(image)
        try:
            faces= DeepFace.analyze(img, detector_backend=detector, enforce_detection=True, expand_percentage=30)
            for face in faces:
                x, y, width, height, left_eye, right_eye = face["region"].values()
                plt.gca().add_patch(patches.Rectangle((x,y), width, height, linewidth=2, edgecolor=color, facecolor='none'))
                fs=8
                plt.text(x+fs*1.5, y-fs*3, f"{face["face_confidence"]}", color=color, backgroundcolor='black', fontsize=8) 
            plt.imshow(img)
            plt.show()
            for face in faces:
                x, y, width, height, left_eye, right_eye = face["region"].values()
                face_img= img[y:y+height,x:x+width]
                pimage= Image.fromarray(face_img)
                h= phash(pimage)
                face_path= extracted_faces.joinpath(f"{h}.jpg")
                if face_path.exists():
                    print("Duplicate")
                else:
                    pimage.save(face_path)
                metadata_path= extracted_faces.joinpath(f"{h}.json")
                if not metadata_path.exists():
                    with metadata_path.open('w') as f:
                        json.dump(face, f, indent=2)
                plt.imshow(face_img)
                plt.show()
                print(f"Confidence {face["face_confidence"]}, Age: {face["age"]} yo, Gender: {face["dominant_gender"]}, Race: {face["dominant_race"]}, Emotion: {face["dominant_emotion"]} phash {h}")
            
        except ValueError:
            plt.imshow(img)
            plt.text(0,0,"No face detected", color=color, backgroundcolor='black', fontsize=8)
            plt.show()
        if index == -1:
            break
    